In [ ]:
# default_exp data.token_classification


In [ ]:
# all_slow


In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.token_classification

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for token classification tasks (e.g., Named entity recognition (NER), Part-of-speech tagging (POS), etc...)

In [ ]:
# export
import os
from typing import List, Tuple

from fastcore.all import *
from fastai.data.block import TransformBlock, Category, CategoryMap
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForTokenClassification, logging, PretrainedConfig, PreTrainedTokenizerBase, PreTrainedModel

from blurr.utils import BLURR
from blurr.data.core import HF_BaseInput, HF_BeforeBatchTransform, first_blurr_tfm

logging.set_verbosity_error()


In [ ]:
# hide_input
import pdb

from datasets import load_dataset
from fastai.data.block import DataBlock, ColReader, ColSplitter
from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbverbose.showdoc import show_doc

from blurr.utils import print_versions
from blurr.data.core import HF_TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")


What we're running with at the time this documentation was generated:
torch: 1.10.1+cu111
fastai: 2.5.3
transformers: 4.15.0


In [ ]:
# hide
# cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Setup

We'll use a subset of `conll2003` to demonstrate how to configure your blurr code for token classification

In [ ]:
raw_datasets = load_dataset("conll2003")
raw_datasets


Reusing dataset conll2003 (/home/wgilliam/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

We need to get a list of the distinct entities we want to predict. If they are represented as list in their raw/readable form in another attribute/column in our dataset, we could use something like this to build a sorted list of distinct values as such: `labels = sorted(list(set([lbls for sublist in germ_eval_df.labels.tolist() for lbls in sublist])))`.

Fortunately, the `conll2003` dataset allows us to get at this list directly using the code below.

In [ ]:
labels = raw_datasets["train"].features["ner_tags"].feature.names
labels


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
#hide
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])
print([(word, labels[label_idx]) for word, label_idx in zip(raw_datasets["train"][0]["tokens"], raw_datasets["train"][0]["ner_tags"])])


['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]
[('EU', 'B-ORG'), ('rejects', 'O'), ('German', 'B-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'B-MISC'), ('lamb', 'O'), ('.', 'O')]


In [ ]:
conll2003_df = pd.DataFrame(raw_datasets["train"])


In [ ]:
model_cls = AutoModelForTokenClassification

pretrained_model_name = "roberta-base" # "bert-base-multilingual-cased"
n_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(
    pretrained_model_name, model_cls=model_cls, config_kwargs={"num_labels": n_labels}
)

hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)


('roberta',
 transformers.models.roberta.configuration_roberta.RobertaConfig,
 transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast,
 transformers.models.roberta.modeling_roberta.RobertaForTokenClassification)

## Mid-level API

In [ ]:
# export
class HF_TokenTensorCategory(TensorBase):
    pass


In [ ]:
# export
class HF_TokenCategorize(Transform):
    """Reversible transform of a list of category string to `vocab` id"""

    def __init__(
        self,
        # The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))
        vocab=None,
        # The token used to identifiy ignored tokens (default: xIGNx)
        ignore_token=None,
        # The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)
        ignore_token_id=None,
    ):
        self.vocab = None if vocab is None else CategoryMap(vocab, sort=False)
        self.ignore_token = "[xIGNx]" if ignore_token is None else ignore_token
        self.ignore_token_id = CrossEntropyLossFlat().ignore_index if ignore_token_id is None else ignore_token_id

        self.loss_func, self.order = CrossEntropyLossFlat(ignore_index=self.ignore_token_id), 1

    def setups(self, dsets):
        if self.vocab is None and dsets is not None:
            self.vocab = CategoryMap(dsets)
        self.c = len(self.vocab)

    def encodes(self, labels):
        # if `val` is the label name (e.g., B-PER, I-PER, etc...), lookup the corresponding index in the vocab using
        # `self.vocab.o2i`
        ids = [val if (isinstance(val, int)) else self.vocab.o2i[val] for val in labels]
        return HF_TokenTensorCategory(ids)

    def decodes(self, encoded_labels):
        return Category([(self.vocab[lbl_id]) for lbl_id in encoded_labels if lbl_id != self.ignore_token_id])


`HF_TokenCategorize` modifies the fastai `Categorize` transform in a couple of ways.  

First, it allows your targets to consist of a `Category` ***per*** token, and second, it uses the idea of an `ignore_token_id` to mask subtokens that don't need a prediction.  For example, the target of special tokens (e.g., pad, cls, sep) are set to `ignore_token_id` as are subsequent sub-tokens of a given token should more than 1 sub-token make it up.

In [ ]:
# export
def HF_TokenCategoryBlock(
    # The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))
    vocab=None,
    # The token used to identifiy ignored tokens (default: xIGNx)
    ignore_token=None,
    # The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)
    ignore_token_id=None,
):
    """`TransformBlock` for per-token categorical targets"""
    return TransformBlock(type_tfms=HF_TokenCategorize(vocab=vocab, ignore_token=ignore_token, ignore_token_id=ignore_token_id))


In [ ]:
show_doc(HF_TokenCategoryBlock)


<h4 id="HF_TokenCategoryBlock" class="doc_header"><code>HF_TokenCategoryBlock</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TokenCategoryBlock</code>(**`vocab`**=*`None`*, **`ignore_token`**=*`None`*, **`ignore_token_id`**=*`None`*)

`TransformBlock` for per-token categorical targets

**Parameters:**


 - **`vocab`** : *`<class 'NoneType'>`*, *optional*	<p>The unique list of entities (e.g., B-LOC) (default: CategoryMap(vocab))</p>


 - **`ignore_token`** : *`<class 'NoneType'>`*, *optional*	<p>The token used to identifiy ignored tokens (default: xIGNx)</p>


 - **`ignore_token_id`** : *`<class 'NoneType'>`*, *optional*	<p>The token ID that should be ignored when calculating the loss (default: CrossEntropyLossFlat().ignore_index)</p>



Again, we define a custom class, `HF_TokenClassInput`, for the @typedispatched methods to use so that we can override how token classification inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
# export
class HF_TokenClassInput(HF_BaseInput):
    pass


In [ ]:
# export
class HF_TokenClassBeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(
        self,
        # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
        hf_arch: str,
        # A specific configuration instance you want to use
        hf_config: PretrainedConfig,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # A Hugging Face model
        hf_model: PreTrainedModel,
        # The token ID that should be ignored when calculating the loss
        ignore_token_id=CrossEntropyLossFlat().ignore_index,
        # Labeling strategy (defaults to replacing each token with it's related entity's label)
        label_strategy: str = "replace_tokens_with_same_label",
        # To control the length of the padding/truncation. It can be an integer or None,
        # in which case it will default to the maximum length the model can accept. If the model has no
        # specific maximum input length, truncation/padding to max_length is deactivated.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        max_length: int = None,
        # To control the `padding` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `'do_not_pad'.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        padding: Union[bool, str] = True,
        # To control `truncation` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `do_not_truncate`.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        truncation: Union[bool, str] = True,
        # The `is_split_into_words` argument applied to your `hf_tokenizer` during tokenization. Set this to `True`
        # if your inputs are pre-tokenized (not numericalized)
        is_split_into_words: bool = True,
        # Any other keyword arguments you want included when using your `hf_tokenizer` to tokenize your inputs
        tok_kwargs={},
        # Keyword arguments to apply to `HF_TokenClassBeforeBatchTransform`
        **kwargs
    ):
        tok_kwargs = {**tok_kwargs, **{"return_special_tokens_mask": True}}

        super().__init__(
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            ignore_token_id=ignore_token_id,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            is_split_into_words=is_split_into_words,
            tok_kwargs=tok_kwargs,
            **kwargs
        )

        if label_strategy == "replace_tokens_with_same_label":
            self.label_strategy_func = self._replace_tokens_with_same_label_strategy
        else:
            raise NotImplementedError()

    def encodes(self, samples):
        samples, batch_encoding = super().encodes(samples, return_batch_encoding=True)

        # if there are no targets (e.g., when used for inference), there is no need to do any post-processing
        # on the labels
        if len(samples[0]) == 1:
            return samples

        # get the type of our targets (by default will be HF_TokenTensorCategory)
        target_cls = type(samples[0][1])

        # we assume that first target = the categories we want to predict for each token
        updated_samples = []
        for idx, s in enumerate(samples):
            word_ids = batch_encoding.word_ids(idx)
            targ_ids = self.label_strategy_func(s[1], word_ids, target_cls)
            updated_samples.append((s[0], targ_ids))

        return updated_samples

    def _replace_tokens_with_same_label_strategy(self, word_label_ids, word_ids, trg_class=HF_TokenTensorCategory):
        targ_ids = trg_class([self.ignore_token_id if (word_id == None) else word_label_ids[word_id] for word_id in word_ids])
        return targ_ids



`HF_TokenClassBeforeBatchTransform` is used to exclude any of the target's tokens we don't want to include in the loss calcuation (e.g. padding, cls, sep, etc...). Notice also that we default `is_split_into_words = True` since for most token classification problems, each example comes in the form as a list of words and a list of entity labels.

In [ ]:
before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model)
blocks = (HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), HF_TokenCategoryBlock(vocab=labels))

dblock = DataBlock(blocks=blocks, get_x=ColReader("tokens"), get_y=ColReader("ner_tags"), splitter=RandomSplitter())


In [ ]:
# hide
# dblock.summary(conll2003_df)


In [ ]:
dls = dblock.dataloaders(conll2003_df, bs=4)


In [ ]:
b = dls.one_batch()


In [ ]:
len(b), b[0]["input_ids"].shape, b[1].shape


(2, torch.Size([4, 29]), torch.Size([4, 29]))

In [ ]:
# export
def align_labels_with_tokens(
    # A Hugging Face tokenizer
    hf_tokenizer: PreTrainedTokenizerBase,
    # List of input_ids for the tokens in a single piece of processed text
    input_ids: List[int], 
    # List of label indexs for each token
    token_label_ids, 
    # List of label names from witch the `label` indicies can be used to find the name of the label
    vocab
) -> List[Tuple[str, str]]:
    """
    Given a list of input IDs, the label ID associated to each, and the labels vocab, this method will return a list of tuples whereby
    each tuple defines the "token" and its label name. For example: 
    [('ĠWay', B-PER), ('de', B-PER), ('ĠGill', I-PER), ('iam', I-PER), ('Ġloves'), ('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG)]
    """
    # convert ids to tokens
    toks = hf_tokenizer.convert_ids_to_tokens(input_ids)
    # align "tokens" with labels
    tok_labels = [(tok, vocab[label_id]) for tok, label_id in zip(toks, token_label_ids) if tok not in hf_tokenizer.special_tokens_map.values()]
    return tok_labels


def align_labels_with_words(
    # A Hugging Face tokenizer
    hf_tokenizer: PreTrainedTokenizerBase,
    # A list of tuples, where each represents a token and its label (e.g., [('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG), ...])
    tok_labels
) -> List[Tuple[str, str]]:
    """
    Given a list of tuples where each tuple defines a token and its label, return a list of tuples whereby each tuple defines the
    "word" and its label. Method assumes that model inputs are a list of words, and in conjunction with the `align_labels_with_tokens` method,
    allows the user to reconstruct the orginal raw inputs and labels.
    """
    # recreate raw words list (we assume for token classification that the input is a list of words)
    words = hf_tokenizer.convert_tokens_to_string([tok_label[0] for tok_label in tok_labels]).split()
    word_list = [word for word in words]
    # align "words" with labels
    word_labels, idx = [], 0
    for word in word_list:
        word_labels.append((word, tok_labels[idx][1]))
        idx += len(hf_tokenizer.tokenize(word))

    return word_labels


In [ ]:
for idx in range(5):
    raw_word_list = conll2003_df.iloc[idx]['tokens']
    raw_label_list = conll2003_df.iloc[idx]['ner_tags']

    be = hf_tokenizer(raw_word_list, is_split_into_words=True)
    input_ids = be['input_ids']
    targ_ids = [-100 if (word_id == None) else raw_label_list[word_id] for word_id in be.word_ids()]

    tok_labels = align_labels_with_tokens(hf_tokenizer, input_ids, targ_ids, labels)
    word_labels = align_labels_with_words(hf_tokenizer, tok_labels)

    for word_label, raw_word, raw_label_id in zip (word_labels, raw_word_list, raw_label_list):
        test_eq(word_label[0], raw_word)
        test_eq(word_label[1], labels[raw_label_id])

In [ ]:
show_doc(align_labels_with_tokens)
show_doc(align_labels_with_words)

<h4 id="align_labels_with_tokens" class="doc_header"><code>align_labels_with_tokens</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>align_labels_with_tokens</code>(**`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`input_ids`**:`List`\[`int`\], **`token_label_ids`**, **`vocab`**)

Given a list of input IDs, the label ID associated to each, and the labels vocab, this method will return a list of tuples whereby
each tuple defines the "token" and its label name. For example: 
[('ĠWay', B-PER), ('de', B-PER), ('ĠGill', I-PER), ('iam', I-PER), ('Ġloves'), ('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG)]

**Parameters:**


 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*	<p>A Hugging Face tokenizer</p>


 - **`input_ids`** : *`typing.List[int]`*	<p>List of input_ids for the tokens in a single piece of processed text</p>


 - **`token_label_ids`** : *`<class 'inspect._empty'>`*	<p>List of label indexs for each token</p>


 - **`vocab`** : *`<class 'inspect._empty'>`*	<p>List of label names from witch the `label` indicies can be used to find the name of the label</p>



**Returns**:
	
 * *`typing.List[typing.Tuple[str, str]]`*

<h4 id="align_labels_with_words" class="doc_header"><code>align_labels_with_words</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>align_labels_with_words</code>(**`hf_tokenizer`**:`PreTrainedTokenizerBase`, **`tok_labels`**)

Given a list of tuples where each tuple defines a token and its label, return a list of tuples whereby each tuple defines the
"word" and its label. Method assumes that model inputs are a list of words, and in conjunction with the [`align_labels_with_tokens`](/blurr/data-token-classification.html#align_labels_with_tokens) method,
allows the user to reconstruct the orginal raw inputs and labels.

**Parameters:**


 - **`hf_tokenizer`** : *`<class 'transformers.tokenization_utils_base.PreTrainedTokenizerBase'>`*	<p>A Hugging Face tokenizer</p>


 - **`tok_labels`** : *`<class 'inspect._empty'>`*	<p>A list of tuples, where each represents a token and its label (e.g., [('ĠHug', B-ORG), ('ging', B-ORG), ('ĠFace', I-ORG), ...])</p>



**Returns**:
	
 * *`typing.List[typing.Tuple[str, str]]`*

In [ ]:
# export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `HF_TokenClassInput` typed inputs
    x: HF_TokenClassInput,
    y,
    # Your raw inputs/targets
    samples,
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for
    # decoding them into something understandable
    dataloaders,
    # Your `show_batch` context
    ctxs=None,
    # The maximum number of items to show
    max_n=6,
    # Any truncation your want applied to your decoded inputs
    trunc_at=None,
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs,
):
    # grab our tokenizer
    tfm = first_blurr_tfm(dataloaders, before_batch_tfm_class=HF_TokenClassBeforeBatchTransform)
    hf_tokenizer = tfm.hf_tokenizer
    vocab = dataloaders.vocab

    res = L()
    for inp, trg, sample in zip(x, y, samples):
        # align "tokens" with labels
        tok_labels = align_labels_with_tokens(hf_tokenizer, inp, trg, vocab)
        # align "words" with labels
        word_labels = align_labels_with_words(hf_tokenizer, tok_labels)
        # stringify list of (word,label) for example
        res.append([f"{[ word_targ for idx, word_targ in enumerate(word_labels) if (trunc_at is None or idx < trunc_at) ]}"])

    display_df(pd.DataFrame(res, columns=["word / target label"])[:max_n])
    return ctxs


In [ ]:
dls.show_batch(dataloaders=dls, max_n=5, trunc_at=20)


,word / target label
0,"[('SAN', 'B-ORG'), ('FRANCISCO', 'I-ORG'), ('54', 'O'), ('71', 'O'), ('.432', 'O'), ('14', 'O')]"
1,"[('(', 'O'), ('YMCA', 'B-ORG'), ('Proj', 'O'), ('.', 'O')]"
2,"[('W', 'O'), ('L', 'O'), ('PCT', 'O'), ('GB', 'O')]"
3,"[('U.S.', 'B-LOC'), ('lauds', 'O'), ('Russian-Chechen', 'B-MISC'), ('deal', 'O'), ('.', 'O')]"


## Tests

The tests below to ensure the core DataBlock code above works for **all** pretrained token classification models available in Hugging Face.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained classification models you are working with ... and if any of your pretrained token classification models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
# hide
[model_type for model_type in BLURR.get_models(task="TokenClassification") if (not model_type.startswith("TF"))]



['AlbertForTokenClassification',
 'BertForTokenClassification',
 'BigBirdForTokenClassification',
 'CamembertForTokenClassification',
 'CanineForTokenClassification',
 'ConvBertForTokenClassification',
 'DebertaForTokenClassification',
 'DebertaV2ForTokenClassification',
 'DistilBertForTokenClassification',
 'ElectraForTokenClassification',
 'FNetForTokenClassification',
 'FlaubertForTokenClassification',
 'FunnelForTokenClassification',
 'GPT2ForTokenClassification',
 'IBertForTokenClassification',
 'LayoutLMForTokenClassification',
 'LayoutLMv2ForTokenClassification',
 'LongformerForTokenClassification',
 'MPNetForTokenClassification',
 'MegatronBertForTokenClassification',
 'MobileBertForTokenClassification',
 'RemBertForTokenClassification',
 'RoFormerForTokenClassification',
 'RobertaForTokenClassification',
 'SqueezeBertForTokenClassification',
 'XLMForTokenClassification',
 'XLMRobertaForTokenClassification',
 'XLNetForTokenClassification']

In [ ]:
# hide
pretrained_model_names = [
    "hf-internal-testing/tiny-albert",
    "hf-internal-testing/tiny-bert",
    "hf-internal-testing/tiny-random-big_bird",
    "camembert-base",
    "hf-internal-testing/tiny-random-canine",
    "YituTech/conv-bert-base",
    "hf-internal-testing/tiny-deberta",
    "hf-internal-testing/tiny-random-deberta-v2",
    "hf-internal-testing/tiny-random-distilbert",
    "hf-internal-testing/tiny-electra",
    "google/fnet-base",
    "hf-internal-testing/tiny-random-flaubert",
    "hf-internal-testing/tiny-random-funnel",
    "hf-internal-testing/tiny-random-gpt2",
    "hf-internal-testing/tiny-layoutlm",
    "microsoft/layoutlmv2-base-uncased",
    "hf-internal-testing/tiny-random-longformer",
    "hf-internal-testing/tiny-random-mpnet",
    "kssteven/ibert-roberta-base",
    "anton-l/megatron-11b",                          
    "hf-internal-testing/tiny-random-mobilebert",
    'google/rembert',
    "hf-internal-testing/tiny-random-roformer",
    'google/reformer-enwik8',                        
    "hf-internal-testing/tiny-random-roberta",
    "hf-internal-testing/tiny-random-squeezebert",
    "hf-internal-testing/tiny-random-xlm",
    "hf-internal-testing/tiny-xlm-roberta",
    "hf-internal-testing/tiny-random-xlnet",
]


In [ ]:
# hide
model_cls = AutoModelForTokenClassification
bsz = 2
seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error = None

    print(f"=== {model_name} ===\n")

    tok_kwargs = {"add_prefix_space": True} if 'deberta' in model_name else {}

    try:
        hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, model_cls=model_cls, tokenizer_kwargs=tok_kwargs)
        print(f"architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n")

        # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
        if hf_tokenizer.pad_token is None:
            hf_tokenizer.add_special_tokens({"pad_token": "<pad>"})
            hf_config.pad_token_id = hf_tokenizer.get_vocab()["<pad>"]
            hf_model.resize_token_embeddings(len(hf_tokenizer))

        before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, padding="max_length", max_length=seq_sz)
        blocks = (HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), HF_TokenCategoryBlock(vocab=labels))
        dblock = DataBlock(blocks=blocks, get_x=ColReader("tokens"), get_y=ColReader("ner_tags"), splitter=RandomSplitter())

        dls = dblock.dataloaders(conll2003_df, bs=bsz)
        b = dls.one_batch()

        print("*** TESTING DataLoaders ***\n")
        test_eq(len(b), 2)
        test_eq(len(b[0]["input_ids"]), bsz)
        test_eq(b[0]["input_ids"].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)

        if hasattr(hf_tokenizer, "add_prefix_space"):
            test_eq(hf_tokenizer.add_prefix_space, True)

        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, "PASSED", ""))
        dls.show_batch(dataloaders=dls, max_n=2, trunc_at=10)

    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, "FAILED", err))



=== hf-internal-testing/tiny-albert ===

architecture:	albert
tokenizer:	AlbertTokenizerFast

*** TESTING DataLoaders ***



,word / target label
0,"[('rudyard', 'B-LOC'), ('mt', 'B-LOC'), ('4.54', 'O'), ('dn', 'O'), ('.10', 'O'), ('---', 'O'), ('---', 'O'), ('---', 'O'), ('---', 'O')]"
1,"[('gooch', 'B-PER'), (',', 'O'), ('who', 'O'), ('retired', 'O'), ('from', 'O'), ('test', 'O'), ('cricket', 'O'), ('after', 'O'), ('the', 'O'), ('1994-95', 'O')]"


=== hf-internal-testing/tiny-bert ===

architecture:	bert
tokenizer:	BertTokenizerFast

*** TESTING DataLoaders ***



,word / target label
0,"[('unam', 'B-ORG'), ('3', 'O'), ('0', 'O'), ('0', 'O'), ('3', 'O'), ('2', 'O'), ('6', 'O'), ('0', 'O')]"
1,"[('sheffield', 'B-ORG'), ('wednesday', 'I-ORG'), ('3', 'O'), ('3', 'O'), ('0', 'O'), ('0', 'O'), ('6', 'O'), ('2', 'O'), ('9', 'O')]"


=== hf-internal-testing/tiny-random-big_bird ===

architecture:	big_bird
tokenizer:	BigBirdTokenizerFast

*** TESTING DataLoaders ***



...

,word / target label
0,"[('hansa', 'B-ORG'), ('rostock', 'I-ORG'), ('3', 'O'), ('0', 'O'), ('2', 'O'), ('1', 'O'), ('3', 'O'), ('4', 'O'), ('2', 'O')]"
1,"[('baseball', 'O'), ('-', 'O'), ('major', 'B-MISC'), ('league', 'I-MISC'), ('standings', 'O'), ('after', 'O'), (""thursday's"", 'O'), ('games.', 'O')]"


=== hf-internal-testing/tiny-random-xlm ===

architecture:	xlm
tokenizer:	XLMTokenizer

Could not do one pass in your dataloader, there is something wrong in it
=== hf-internal-testing/tiny-xlm-roberta ===

architecture:	xlm_roberta
tokenizer:	XLMRobertaTokenizerFast

*** TESTING DataLoaders ***



,word / target label
0,"[('Diplomats', 'O'), ('explain', 'O'), ('the', 'O'), ('purpose', 'O'), ('of', 'O'), ('this', 'O'), ('electoral', 'O'), ('engineering', 'O'), ('is', 'O'), ('to', 'O')]"
1,"[('42.', 'O'), ('Slovenia', 'B-LOC'), ('7.77', 'O')]"


=== hf-internal-testing/tiny-random-xlnet ===

architecture:	xlnet
tokenizer:	XLNetTokenizerFast

*** TESTING DataLoaders ***



,word / target label
0,"[('""', 'O'), ('There', 'O'), ('will', 'O'), ('be', 'O'), ('tough', 'O'), ('matches', 'O'), ('but', 'O'), ('I', 'O'), ('hope', 'O'), ('I', 'O')]"
1,"[('Despite', 'O'), ('the', 'O'), ('current', 'O'), ('difficulties', 'O'), (',', 'O'), ('however', 'O'), (',', 'O'), ('analysts', 'O'), ('were', 'O'), ('convinced', 'O')]"


In [ ]:
# hide_input
test_results_df = pd.DataFrame(test_results, columns=["arch", "tokenizer", "model_name", "result", "error"])
display_df(test_results_df)


,arch,tokenizer,model_name,result,error
0,albert,AlbertTokenizerFast,hf-internal-testing/tiny-albert,PASSED,
1,bert,BertTokenizerFast,hf-internal-testing/tiny-bert,PASSED,
2,big_bird,BigBirdTokenizerFast,hf-internal-testing/tiny-random-big_bird,PASSED,
3,camembert,CamembertTokenizerFast,camembert-base,PASSED,
4,canine,CanineTokenizer,hf-internal-testing/tiny-random-canine,FAILED,word_ids() is not available when using Python-based tokenizers
5,convbert,ConvBertTokenizerFast,YituTech/conv-bert-base,PASSED,
6,deberta,DebertaTokenizerFast,hf-internal-testing/tiny-deberta,PASSED,
7,deberta_v2,DebertaV2Tokenizer,hf-internal-testing/tiny-random-deberta-v2,FAILED,word_ids() is not available when using Python-based tokenizers
8,distilbert,DistilBertTokenizerFast,hf-internal-testing/tiny-random-distilbert,PASSED,
9,electra,ElectraTokenizerFast,hf-internal-testing/tiny-electra,PASSED,


## Summary

This module includes all the low, mid, and high-level API bits for token classification tasks data prep.

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()


Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted 99e_examples-causal-lm-gpt2.ipynb.
Converted index.ipynb.
